In [4]:
# Add your import statement
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import mixture
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.cluster import KMeansClusterer, \
cosine_distance
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from textblob import Word
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv("/Users/harshithar/Documents/Assignments/Spring_2023/BIA_686/project/NYC_Jobs.csv") 
df.head()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,533891,HRA/DEPT OF SOCIAL SERVICES,External,1,CONTRACT ANALYST,STAFF ANALYST,Competitive-1,12626,02,"Administration & Human Resources Policy, Resea...",...,**LOAN FORGIVENESS The federal government prov...,APPLICANTS MUST BE PERMANENT IN THE STAFF ANAL...,NaN,NaN,NaN,New York City residency is generally required ...,05/27/2022,NaN,11/07/2022,02/07/2023
1,554785,DEPARTMENT OF TRANSPORTATION,External,1,DESIGN ENGINEER,CIVIL ENGINEERING INTERN,Competitive-1,20202,00,"Engineering, Architecture, & Planning",...,As a current or prospective employee of the Ci...,RESUMES MUST BE SUBMITTED ELECTRONICALLY USING...,NaN,NaN,NaN,New York City Residency is not required for th...,10/12/2022,NaN,12/28/2022,02/07/2023
2,568481,DEPT OF DESIGN & CONSTRUCTION,Internal,1,Director,ADM PROC ANAL-NM FRM M1-3,Competitive-1,8297A,00,"Finance, Accounting, & Procurement",...,NaN,"For City Employees, please go to Employee Self...",35 Hours,"30-30 Thomson Ave, LIC, NY 11101",NaN,New York City residency is generally required ...,01/20/2023,21-MAR-2023,01/20/2023,02/07/2023
3,527833,NYC HOUSING AUTHORITY,External,1,ELECTRICIAN,ELECTRICIAN,Competitive-1,91717,00,Building Operations & Maintenance,...,"1.\tFor NYCHA employees, these positions are o...",Click the Apply Now button.,NaN,NaN,NaN,NYCHA has no residency requirements.,04/13/2022,NaN,05/12/2022,02/07/2023
4,541555,FIRE DEPARTMENT,External,1,"Director, Data Quality",CITY RESEARCH SCIENTIST,Non-Competitive-5,21744,4A,"Technology, Data & Innovation Policy, Research...",...,***LOAN FORGIVENESS: As a current or prospecti...,NON-CITY EMPLOYEES/EXTERNAL CANDIDATES PLEASE ...,NaN,NaN,NaN,New York City residency is generally required ...,08/10/2022,NaN,08/10/2022,02/07/2023


In [5]:
##to string
df['Job Description'] = df['Job Description'].apply(lambda x: str(x))
## Lower case
df['Job Description'] = df['Job Description'].apply(lambda x: " ".join(x.lower()for x in x.split()))
## remove special charecters
df['Job Description'] = df['Job Description'].str.replace('[^a-zA-Z0-9]', ' ')
## digits
df['Job Description'] = df['Job Description'].str.replace('\d+', '')

#remove stop words
stop = stopwords.words('english')
df['Job Description'] = df['Job Description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

## lemmatization
df['Job Description'] = df['Job Description'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

print("Preprocessed data: \n")
print(df['Job Description'])

<ipython-input-5-64eafdec983e>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Job Description'] = df['Job Description'].str.replace('[^a-zA-Z0-9]', ' ')
<ipython-input-5-64eafdec983e>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Job Description'] = df['Job Description'].str.replace('\d+', '')


Preprocessed data: 

0       department social service ds comprised adminis...
1       division sidewalk inspection management sim re...
2       permanent administrative procurement analyst t...
3       direction electrician supervisor electrician a...
4       fire department city new york fdny largest fir...
                              ...                        
5929    nyc department environmental protection dep en...
5930    new york city home approximately million older...
5931    open candidate permanent administrative procur...
5932    bronx district attorney office seeking well qu...
5933    ds office audit service oas serf liaison exter...
Name: Job Description, Length: 5934, dtype: object


In [10]:

#Tokenize and build vocabulary
X = df['Job Description']
y = df['Business Title']

# split data into 80% training and 20% test
train_text, test_text, train_label, test_label = train_test_split(X, y, test_size=0.2,random_state=109) 


In [16]:
tfidf_vect = TfidfVectorizer(stop_words="english",min_df=2) 

dtm= tfidf_vect.fit_transform(train_text)
    
num_clusters=20
    
clusterer = KMeansClusterer(num_clusters, cosine_distance, repeats=5)
clusters_cosine = clusterer.cluster(dtm.toarray(), assign_clusters=True)
centroids=np.array(clusterer.means())
sorted_centroids = centroids.argsort()[:, ::-1] 
voc_lookup= tfidf_vect.get_feature_names_out()
for i in range(num_clusters):
        top_words=[voc_lookup[word_index] \
               for word_index in sorted_centroids[i, :20]]
        print("Cluster %d:\n %s " % (i, "; ".join(top_words)))

Cluster 0:
 payroll; audit; fisa; opa; tax; financial; timekeeping; agency; management; accounting; mainframe; service; administration; payment; office; city; work; employee; report; data 
Cluster 1:
 claim; oath; snap; hearing; overpayment; administrative; law; cr; trial; city; case; commission; settlement; revenue; judge; office; division; agency; matter; sanction 
Cluster 2:
 attorney; legal; case; law; court; file; bronx; litigation; trial; district; office; counsel; document; agency; witness; discovery; victim; dcwp; work; division 
Cluster 3:
 community; program; agency; training; office; director; outreach; service; project; medium; policy; city; commissioner; staff; development; work; new; support; meeting; communication 
Cluster 4:
 health; child; care; mental; bureau; control; patient; environmental; disease; service; program; injury; social; community; public; clinic; education; new; hygiene; treatment 
Cluster 5:
 energy; city; police; dem; agency; project; york; dcas; new;

In [17]:
tfidf_vect = TfidfVectorizer(stop_words="english",min_df=2) 

dtm= tfidf_vect.fit_transform(df['Business Title'])
    
num_clusters=10
    
clusterer = KMeansClusterer(num_clusters, cosine_distance, repeats=5)
clusters_cosine = clusterer.cluster(dtm.toarray(), assign_clusters=True)
centroids=np.array(clusterer.means())
sorted_centroids = centroids.argsort()[:, ::-1] 
voc_lookup= tfidf_vect.get_feature_names_out()
for i in range(num_clusters):
        top_words=[voc_lookup[word_index] \
               for word_index in sorted_centroids[i, :20]]
        print("Cluster %d:\n %s " % (i, "; ".join(top_words)))

Cluster 0:
 senior; developer; administrator; architect; advisor; application; systems; net; mainframe; web; cook; database; security; engineer; forester; certified; coordinator; writer; clearance; landscape 
Cluster 1:
 assistant; engineer; coordinator; attorney; counsel; aide; supervisor; civil; administrative; college; intern; agency; police; general; investigator; worker; management; engineering; paralegal; case 
Cluster 2:
 manager; project; accountable; senior; case; division; tenant; associate; assistant; resources; operations; office; coordinator; engineer; construction; finance; protective; contract; child; systems 
Cluster 3:
 analyst; data; business; staff; budget; senior; contract; compliance; procurement; operations; research; policy; supervising; management; bureau; payment; scientist; systems; programmer; capital 
Cluster 4:
 energy; examiner; plan; manager; code; clean; project; assistant; analyst; deputy; claims; senior; inspector; program; efficiency; field; director;